In [ ]:
%reload_ext autoreload
%autoreload 2

import time

start = time.time()
from torch_snippets import *
end = time.time()
print(end-start)

4.478874921798706

In [ ]:
from torch_snippets.misc import Timer
import time

n = 5
x = Timer(n)
for i in range(n):
    time.sleep(0.2)
    x()

In [ ]:
%%writefile tmp.txt
a
batchify
callable
dcopy

Writing tmp.txt


In [ ]:
P('tmp.txt').rm('you want to remove {self}? [y/N]')

you want to remove tmp.txt? [y/N]y


In [ ]:
from torch_snippets.ipython import backup_this_notebook

backup_this_notebook('testing.ipynb', '/tmp/testing')

[03/05/23 00:34:00] INFO     Backing up this version of notebook to              ]8;id=880200;file:///Users/yeshwanth.y/code/torch_snippets/torch_snippets/ipython.py\ipython.py]8;;\:]8;id=613159;file:///Users/yeshwanth.y/code/torch_snippets/torch_snippets/ipython.py#backup_this_notebook:43\backup_this_notebook:43]8;;\
                             /tmp/testing/0001.html                                                                